In [5]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores.chroma import Chroma
from langchain.storage import  LocalFileStore
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough



llm = ChatOpenAI(
    temperature=0.1
)

cache_dir = LocalFileStore("./cache")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n"
)


loader = UnstructuredFileLoader("./files/chapter1.txt")

docs = loader.load_and_split(text_splitter=splitter)

embedder = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embedder, cache_dir
)


vectorstore = Chroma.from_documents(docs,cached_embeddings)

retriver= vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            'system',
            """너는 소설을 설명해주는 도우미야. 답변은 반드시 알고있는 것만으로 해줘.
            만약 네가 모르는 것을 물어보면, 모른다고 대답해도 돼.
            """
        ),
        (
            'human',
            '{question}'
        )

    ]
)

chain = (
    {
        'context': retriver,
        'question': RunnablePassthrough()
    }
    | prompt
    | llm
)

chain.invoke('려은은 누구야?')

AIMessage(content='려은은 소설 속의 주인공이 아니기 때문에, 누구인지에 대해서는 알지 못해요. 죄송해요.')